In [ ]:
# default_exp main
from nbdev import *

# Main

> CLI for the PyTorch neural style transfer application

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from time import time

import click
import torchvision.models as models

from pytorch_nst.config import device
from pytorch_nst.util import image_loader, imshow, random_img, save_image, show_all_images
from pytorch_nst.nst import run_style_transfer

In [ ]:
#export
def validate_layers(ctx, param, value):
    try:
        layers = [l for l in value.split(',')]
        convs = [(int(p),int(l)) for p,l in [layer.split("_") for layer in layers]]
        assert all(isinstance(l, int) and isinstance(p, int) for p,l in convs)
        return [f'conv{l}' for l in layers]
    except ValueError:
        raise click.BadParameter('Layers need to be ints seperated by commas')

In [ ]:
#export
@click.command()
@click.option('-c', '--content', 
                default='./examples/content.jpg', 
                prompt='Content image',
                help='path to content image',
                show_default=True)
@click.option('-s', '--style', 
                default='./examples/style.jpg', 
                prompt='Style image',
                help='path to style image',
                show_default=True)
@click.option('-o', '--output', 
                default='./output/generated-' +  str(int(time())) + '.jpg', 
                prompt=False,
                help='path to save generated image',
                show_default=True)
@click.option('--style_weight', default=1000000, prompt=False, show_default=True)
@click.option('--style_layers', callback=validate_layers, 
                default='1_1,2_1,3_1,4_1,5_1', show_default=True,
                help='Conv Layers to use for style loss')
@click.option('--steps', default=300, prompt=False, show_default=True)
@click.option('--random_input', is_flag=True,
                help='Will start with random noise if set, otherwise content image')
def cli(content, style, output, style_weight, style_layers, steps, random_input):
    """Command Line interface for running neural style transfer with PyTorch"""
    style_img = image_loader(style)
    content_img = image_loader(content)
    assert style_img.size() == content_img.size(), \
        "Style and Content images must be the same size"\

    if random_input:
        input_img = random_img()
    else:
        input_img = content_img.clone()

    # Load a pre-trained VGG network
    print("Loading pre-trained VGG19. This may take a while the first run...")
    
    vgg = models.vgg19(pretrained=True).features #Conv2D and pooling layers
    
    #for param in vgg.parameters():
    #    param.requires_grad_(False)
        
    vgg.to(device).eval()
    #cnn = models.resnet50(pretrained=True).to(device).eval()

    gen_img = run_style_transfer(vgg, 
                                 content_img, 
                                 style_img, 
                                 input_img, 
                                 style_layers=style_layers, 
                                 style_weight=style_weight, 
                                 num_steps=steps)

    show_all_images(content_img, style_img, gen_img, title="Generated Image")

    if click.confirm('Do you want to save the generated image?'):
        save_image(gen_img, output)
        print(f'Saved to {output}')

In [ ]:
#hide
show_doc(cli)
show_doc(validate_layers)